# Importing Librabies

---



In [ ]:
#importing Print function
from __future__ import print_function

#Importing Keras
import tensorflow.keras

#Importing Layers from Keras
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten, GlobalAveragePooling2D

#Importing Adam Optimizer
from tensorflow.keras.optimizers import Adam

#Importing ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Importing Model Check Point for saving best MOdel
from keras.callbacks import ModelCheckpoint

#Importing Model
from tensorflow.keras.models import Model

**Defining Parameters**

In [ ]:
Batch_Size = 32
Num_of_Iterations = 50
Num_of_Classes = 5
Num_of_Channels = 3
image_dimn = [32,32]
input_image_shape = [32,32,3]

# Loading Data

---



In [ ]:
!7z x Training.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 141486870 bytes (135 MiB)

Extracting archive: Training.zip
 81% 4096 Open              --
Path = Training.zip
Type = zip
Physical Size = 141486870

  0%    
Would you like to replace the existing file:
  Path:     ./Trainingless/C/982.jpg
  Size:     22472 bytes (22 KiB)
  Modified: 2021-11-15 02:31:22
with the file from archive:
  Path:     Trainingless/C/982.jpg
  Size:     22472 bytes (22 KiB)
  Modified: 2021-11-15 02:31:22
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% - Trainingless/C/982.jpg                               7% 371 - Trainingless/E/940.jpg  

In [ ]:
!7z x Testing.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 39331846 bytes (38 MiB)

Extracting archive: Testing.zip
--
Path = Testing.zip
Type = zip
Physical Size = 39331846

  0%    
Would you like to replace the existing file:
  Path:     ./Testless/D/250.jpg
  Size:     22859 bytes (23 KiB)
  Modified: 2021-11-15 03:33:06
with the file from archive:
  Path:     Testless/D/250.jpg
  Size:     22859 bytes (23 KiB)
  Modified: 2021-11-15 03:33:06
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% - Testless/D/250.jpg                          51% 600 - Testless/B/131.jpg                              95% 1170 - Testless/C/116.jpg

**Data Flow from Directories**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1/255, 
)

train_generator = train_datagen.flow_from_directory(
    directory="/content/Trainingless",
    target_size=image_dimn,
    color_mode="rgb",
    batch_size=Batch_Size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 5000 images belonging to 5 classes.


In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1/255,
)

test_generator = test_datagen.flow_from_directory(
    directory="/content/Testless",
    target_size=image_dimn,
    color_mode="rgb",
    batch_size=Batch_Size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 1250 images belonging to 5 classes.


# Building Model Using Transfer Learning (PreTrained ResNet50)

---



**Building our Model using PreTrained ResNet50 as Backbone**

In [ ]:
#Import PreTrained ResNet50 Model from Keras applications
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

#This pretrained model is used as backbone for our Model
base_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=input_image_shape
)

#From the output of pretrained Model, add Fully connected Dense Layers for Classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

#Defining Inputs and Outputs of our Model
model = Model(inputs=base_model.input, outputs=predictions)

#Because we don't want to again train the pretrained Model, we freeze the layers
for layer in base_model.layers:
    layer.trainable = False

94781440/94765736 [==============================] - 1s 0us/step


**Compiling the Model**

In [ ]:
#Compiling the model using adam as optimizer, categorical_crossentropy as loss and accuracy as Metric
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

**Training the Model**

In [ ]:
#Path for storing the Model Check point
checkpoint_filepath = '/content/sample_data/'

#callbacks to save the model that has best validation accuracy
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#Start training the model
hist = model.fit(train_generator, validation_data=(test_generator), batch_size = Batch_Size, epochs = 100, shuffle=True, callbacks=[model_checkpoint_callback])

#Load weights to the model from the best saved file
model.load_weights(checkpoint_filepath)

Epoch 1/100
157/157 [==============================] - 49s 107ms/step - loss: 1.2832 - accuracy: 0.4508 - val_loss: 1.0334 - val_accuracy: 0.5264
Epoch 2/100
157/157 [==============================] - 15s 94ms/step - loss: 0.8343 - accuracy: 0.6400 - val_loss: 0.6986 - val_accuracy: 0.7608
Epoch 3/100
157/157 [==============================] - 12s 76ms/step - loss: 0.7768 - accuracy: 0.6552 - val_loss: 0.7786 - val_accuracy: 0.6408
Epoch 4/100
157/157 [==============================] - 12s 78ms/step - loss: 0.7255 - accuracy: 0.6738 - val_loss: 0.7274 - val_accuracy: 0.6096
Epoch 5/100
157/157 [==============================] - 12s 79ms/step - loss: 0.6885 - accuracy: 0.6928 - val_loss: 0.8578 - val_accuracy: 0.6344
Epoch 6/100
157/157 [==============================] - 12s 79ms/step - loss: 0.6572 - accuracy: 0.7084 - val_loss: 0.6379 - val_accuracy: 0.7576
Epoch 7/100
157/157 [==============================] - 16s 100ms/step - loss: 0.6353 - accuracy: 0.7184 - val_loss: 0.6484 - val_

**Testing the Model**

In [ ]:
#Evaluate our model on the test data
scores = model.evaluate(test_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', 100*scores[1])

40/40 [==============================] - 3s 62ms/step - loss: 0.4696 - accuracy: 0.8160
Test loss: 0.4695870578289032
Test accuracy: 81.5999984741211


# Model Accuracy : 81.59 %
